# VMEC++ Based Sampling Demo
This notebook demonstrates how to sample an omnigenous field and targets, run a boundary optimization, and visualize the resulting field in Boozer coordinates.

In [ ]:
from constellaration.omnigeneity.omnigenity_field_sampling import (
    SampleOmnigenousFieldAndTargetsSettings,
)
from constellaration.data_generation.vmec_optimization_settings import (
    OmnigenousFieldVmecOptimizationSettings,
)
from constellaration.data_generation import vmec_optimization
from constellaration.mhd import (
    vmec_utils,
    vmec_settings as vmec_settings_module,
    ideal_mhd_parameters as ideal_mhd_parameters_module,
)
from constellaration.boozer import boozer as boozer_module
from constellaration.utils import visualization
from constellaration.omnigeneity import omnigenity_field_plot

## 1. Sample an Omnigenous Field and Targets

In [ ]:
# Sample a single omnigenous field and targets
sampler = SampleOmnigenousFieldAndTargetsSettings(n_samples=1)
sample = sampler.sample_omnigenous_fields_and_targets()[0]

# Inspect sample
print(sample.model_dump_json(indent=4))

## 2. Run VMEC Boundary Optimization

In [ ]:
# Run for ~2m as an example
settings = OmnigenousFieldVmecOptimizationSettings(
    max_poloidal_mode=2,
    max_toroidal_mode=2,
    gradient_free_optimization_hypercube_bounds=0.33,
    gradient_free_max_time=120,
    verbose=True,
)
optimized_surface = vmec_optimization.optimize_boundary_omnigenity_vmec(
    sample, settings
)

## 3. Plot the resulting boundary and its field in Boozer coordinates

In [ ]:
visualization.plot_boundary(optimized_surface)

In [ ]:
# Compute the equilibrium
vmec_preset_settings = vmec_settings_module.VmecPresetSettings(
    fidelity="low_fidelity",
)
vmec_settings = vmec_settings_module.create_vmec_settings_from_preset(
    optimized_surface,
    settings=vmec_preset_settings,
)
ideal_mhd_parameters = ideal_mhd_parameters_module.boundary_to_ideal_mhd_parameters(
    optimized_surface
)
equilibrium = vmec_utils.run_vmec(
    boundary=optimized_surface,
    mhd_parameters=ideal_mhd_parameters,
    vmec_settings=vmec_settings,
)

In [ ]:
# Visualize the target field
omnigenity_field_plot.plot_boozer_field(sample.omnigenous_field)

# Visualize the optimized field
boozer_settings = boozer_module.BoozerSettings(normalized_toroidal_flux=[1])
final_field_figure = visualization.plot_boozer_surfaces(
    equilibrium=equilibrium,
    settings=boozer_settings,
)[0]
final_field_figure.show()